# import

In [ ]:
import pandas as pd
import numpy as np
import requests
import folium

# 크롤링
import time
from bs4 import BeautifulSoup

!pip install selenium

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


# 크롤링 및 카카오 API

## 스타벅스 크롤링

In [ ]:
# 크롬드라이버 실행
driver = webdriver.Chrome()

#크롬 드라이버에 url 주소 넣고 실행
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# 스타벅스 서울 클릭
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,starbucks_seoul_css ))).click()

# 스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,starbucks_seoul_all_css ))).click()

time.sleep(3)

#스타벅스 서울 전체 HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")

container = soup.find("div", id = "mCSB_3_container")
li_all = container.find_all("li")

starbucks_data = []

for li in li_all:
    name = li.find("strong").text.strip()
    address = li.find("p").text.strip().replace("1522-3232","")
    gu = address.split(" ")[1]
    each = {"매장이름":name, "주소":address, "구":gu}
    starbucks_data.append(each)

df_starbucks = pd.DataFrame(starbucks_data)

## 카카오 API

In [ ]:
# 카카오 API 이용해 주소 변환(도로명 -> 지번주소)
def road_to_dong(df_name, column):
  # 도로명 주소 가져오기
  addrs = df_name[column]

  # 카카오맵 API로 도로명 주소 가져오기
  locations = []
  for addr in addrs:
      url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
      headers = {'Authorization': 'KakaoAK ' + 'c20337c1708cfbd425e6b94f0e5e53cb'}
      place = requests.get(url, headers = headers).json()['documents']
      locations.append(place)

  # 예외 처리하기
  total = [] ## 전체 주소
  dong = [] ## 동
  for i in range(len(locations)):
      try:
          total.append(locations[i][0].get('address').get('address_name'))
          dong.append(locations[i][0].get('address').get('region_3depth_name'))

      except IndexError:
          print(i,'번째 주소 못가져옴', end ='')
          print()
          total.append('없음')
          dong.append('없음')

      except AttributeError:
          total.append(locations[i][0].get('road_address').get('address_name'))
          dong.append(locations[i][0].get('road_address').get('region_3depth_name'))

  # 가져온 데이터 확인하기
  total_T = np.array(total).T
  dong_T = np.array(dong).T
  df_name['지번주소'] = total_T
  df_name['동'] = dong_T
  return '주소변환완료'

In [ ]:
# 카카오 API 이용해 주소 변환(좌표 -> 지번주소)
def xy_to_dong(df_name, x좌표, y좌표):
  locations = []
  for i in range(len(df)):
      x = df_name.loc[i, x좌표].replace('\ufeff', '')  # '\ufeff' 제거
      y = df_name.loc[i, y좌표].replace('\ufeff', '')  # '\ufeff' 제거

      x = float(x)
      y = float(y)

      url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={}&y={}'.format(x, y)
      headers = {'Authorization': 'KakaoAK ' + 'c20337c1708cfbd425e6b94f0e5e53cb'}
      place = requests.get(url, headers = headers).json()['documents']
      locations.append(place)


  # 예외 처리하기
  total = [] ## 전체 주소
  dong = [] ## 동
  for i in range(len(locations)):
      try:
          total.append(locations[i][0].get('address').get('address_name'))
          dong.append(locations[i][0].get('address').get('region_3depth_name'))

      except IndexError:
          print(i,'번째 주소 못가져옴', end ='')
          print()
          total.append('없음')
          dong.append('없음')

      except AttributeError:
          total.append(locations[i][0].get('road_address').get('address_name'))
          dong.append(locations[i][0].get('road_address').get('region_3depth_name'))

  # 가져온 데이터 확인하기
  total_T = np.array(total).T
  dong_T = np.array(dong).T
  df_name['지번주소'] = total_T
  df_name['동'] = dong_T
  df_name['동'] = dong_T
  return '주소변환완료'

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 법정동 주소가 없는 데이터(결측치 처리 전)
df_park=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/21년 서울시 공원.csv', encoding='cp949')
df_mart=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 마트.csv', encoding='cp949')
df_bus=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 버스정류장.csv')
df_child=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 어린이집.csv', encoding='cp949')
df_cctv=pd.read_excel('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 CCTV 현황.xlsx')
df_university=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/16년 서울시 대학 및 전문대학.csv', encoding='cp949')
df_hospital=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 종합병원.csv')
df_school=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 학교(초중고).csv', encoding='cp949')

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 법정동 변환 및 전처리

In [ ]:
# 공원
## 법정동 변환
road_to_dong(df_park, '공원주소')

# 마트
## 영업상태, 업태구분으로 필터링
df_mart.drop(index=df_mart[df_mart['영업상태명']!='영업/정상'].index, inplace=True)
df_mart.drop(index=df_mart[df_mart['업태구분명']=='구분없음'].index, inplace=True)
df_mart.drop(index=df_mart[df_mart['업태구분명']=='그 밖의 대규모점포'].index, inplace=True)
df_mart.drop(index=df_mart[df_mart['사업장명']=='롯데슈퍼하월곡점'].index, inplace=True)
## 지번주소 동 분리
df_mart[['시', '구', '동', '기타']] = df_mart['지번주소'].str.split(' ', n = 3, expand=True)
df_mart.drop(['시', '기타'], axis=1, inplace=True)
## 사라진 시장 제외
df_mart=df_mart.drop([78, 472,473,620,896])

# 버스
## 법정동 변환
xy_to_dong(df_bus, '공원주소')
# 가상 정류장 제외
df_bus = df_bus.drop(df_bus[df_bus['정류소타입'] == '가상정류장'].index)
# 경기도 지역 제외
df_bus = df_bus.drop([1216,3289,3890,6708,6974,7385,7386,7587,7694,10633,10672,10702,10724,10887,10888,10996])

# 어린이집
## 운영현황 필터링
df_child = df_child.drop(df_child[df_child['운영현황'] == '폐지'].index)
df_child = df_child.drop(df_child[df_child['운영현황'] == '휴지'].index)
## 법정동 주소 추가
road_to_dong(df_child, '상세주소')
## 운영중단 어린이집 제외
df_child = df_child.drop([173, 4911, 4912, 4913, 4914, 4915])

# CCTV
## 법정동 변환
road_to_dong(df_cctv, '소재지도로명주소')

# 스타벅스
## 법정동 변환
road_to_dong(df_starbucks, '주소')

# 지하철
## 지번주소 동 분리
df_subway['동'] = df_subway['기본주소'].str.split(" ",expand=True)[2]

# 대학교
## 법정동 변환
road_to_dong(df_university, '주소')

# 병원
## 법정동 변환
road_to_dong(df_hospital, '주소')

# 초중고
## 법정동 변환
road_to_dong(df_school, '도로명주소')

## 학교종류 필터링
df_school.drop(index=df_school[df_school['학교종류명']=='특수학교'].index, inplace=True)
df_school.drop(index=df_school[df_school['학교종류명']=='공동실습소'].index, inplace=True)
df_school.drop(index=df_school[(df_school['학교종류명'] == '특수학교') |
                              (df_school['학교종류명'] == '평생학교(고)-2년6학기') |
                              (df_school['학교종류명'] == '평생학교(고)-3년6학기') |
                              (df_school['학교종류명'] == '평생학교(중)-2년6학기') |
                              (df_school['학교종류명'] == '평생학교(중)-3년6학기') |
                              (df_school['학교종류명'] == '평생학교(초)-3년6학기')].index, inplace=True)

# 결측치 처리

In [ ]:
# 공원
df_park.loc[8,'동'] = '응봉동'
df_park.loc[9,'동'] = '등촌동'
df_park.loc[21,'동'] = '동숭동'
df_park.loc[25,'동'] = '독산동'
df_park.loc[30,'동'] = '신정동'
df_park.loc[36,'동'] = '사당동'
df_park.loc[38,'동'] = '대방동'
df_park.loc[43,'동'] = '마천동'
df_park.loc[50,'동'] = '여의도동'
df_park.loc[53,'동'] = '방화동'
df_park.loc[58,'동'] = '홍은동'
df_park.loc[65,'동'] = '명륜동'
df_park.loc[68,'동'] = '진관동'
df_park.loc[78,'동'] = '성북동'
df_park.loc[84,'동'] = '잠실동'
df_park.loc[106,'동'] = '사직동'
df_park.loc[110,'동'] = '구산동'
df_park.loc[113,'동'] = '봉천동'
df_park.loc[122,'동'] = '둔총동'
df_park.loc[124,'동'] = '망우동'
df_park.loc[127,'동'] = '면목동'
df_park.loc[129,'동'] = '공릉동'
df_park.loc[131,'동'] = '봉래동'

In [ ]:
# 마트
df_mart.loc[7,'동'] = '망우동'
df_mart.loc[15,'동'] = '망우동'
df_mart.loc[35,'동'] = '영등포동'
df_mart.loc[43,'동'] = '고척동'
df_mart.loc[125,'동'] = '응암동'
df_mart.loc[130,'동'] = '장사동'
df_mart.loc[139,'동'] = '예지동'
df_mart.loc[141,'동'] = '남대문로2가'
df_mart.loc[145,'동'] = '서린동'
df_mart.loc[169,'동'] = '을지로6가'
df_mart.loc[206,'동'] = '남가좌동'
df_mart.loc[206,'동'] = '남가좌동'
df_mart.loc[230,'동'] = '용답동'
df_mart.loc[236,'동'] = '서초동'
df_mart.loc[293,'동'] = '용두동'
df_mart.loc[301,'동'] = '제기동'
df_mart.loc[301,'동'] = '제기동'
df_mart.loc[354,'동'] = '보문동4가'
df_mart.loc[357,'동'] = '장위동'
df_mart.loc[368,'동'] = '길음동'
df_mart.loc[382,'동'] = '장위동'
df_mart.loc[404,'동'] = '수유동'
df_mart.loc[430,'동'] = '남가좌'
df_mart.loc[443,'동'] = '중계동'
df_mart.loc[462,'동'] = '응암동'
df_mart.loc[519,'동'] = '망원동'
df_mart.loc[551,'동'] = '목동'
df_mart.loc[552,'동'] = '신월동'
df_mart.loc[553,'동'] = '목동'
df_mart.loc[554,'동'] = '신월동'
df_mart.loc[576,'동'] = '방화동'
df_mart.loc[644,'동'] = '구로동'
df_mart.loc[713,'동'] = '대방동'
df_mart.loc[940,'동'] = '명일동'
df_mart.loc[941,'동'] = '암사동'
df_mart.loc[943,'동'] = '신천동'

In [ ]:
# 버스
df_bus.loc[442, '동'] = '인현동'
df_bus.loc[1217, '동'] = '금호동1가'
df_bus.loc[1417, '동'] = '행당동'
df_bus.loc[2828, '동'] = '길음동'
df_bus.loc[2956, '동'] = '삼선동'
df_bus.loc[3185, '동'] = '우이동'
df_bus.loc[4323, '동'] = '진관동'
df_bus.loc[4684, '동'] = '녹번동'
df_bus.loc[5576, '동'] = '공덕동'
df_bus.loc[6596, '동'] = '화곡동'
df_bus.loc[7029, '동'] = '항동'
df_bus.loc[7857, '동'] = '대방동'
df_bus.loc[7974, '동'] = '영등포동'
df_bus.loc[10633, '동'] = '동탄8동'
df_bus.loc[10634, '동'] = '거여동'
df_bus.loc[10667, '동'] = '가락동'
df_bus.loc[10675, '동'] = '거여동'
df_bus.loc[10719, '동'] = '거여동'
df_bus.loc[11081, '동'] = '성내동'

In [ ]:
# 어린이집
df_child.loc[26,'동'] = '목동'
df_child.loc[50,'동'] = '길동'
df_child.loc[127,'동'] = '한강로2가'
df_child.loc[210,'동'] = '상암동'
df_child.loc[529,'동'] = '여의도동'
df_child.loc[538,'동'] = '마곡동'
df_child.loc[588,'동'] = '신정동'
df_child.loc[601,'동'] = '장지동'
df_child.loc[617,'동'] = '월계동'
df_child.loc[659,'동'] = '녹번동'
df_child.loc[686,'동'] = '홍은동'
df_child.loc[785,'동'] = '북아현동'
df_child.loc[868,'동'] = '영등포동7가'
df_child.loc[896,'동'] = '상암동'
df_child.loc[898,'동'] = '도화동'
df_child.loc[990,'동'] = '석관동'
df_child.loc[1012,'동'] = '정릉동'
df_child.loc[1015,'동'] = '장위동'
df_child.loc[1021,'동'] = '돈암동'
df_child.loc[1026,'동'] = '종암동'
df_child.loc[1035,'동'] = '종암동'
df_child.loc[1044,'동'] = '창전동'
df_child.loc[1045,'동'] = '정릉동'
df_child.loc[1056,'동'] = '정릉동'
df_child.loc[1068,'동'] = '종암동'
df_child.loc[1073,'동'] = '종암동'
df_child.loc[1086,'동'] = '상일동'
df_child.loc[1102,'동'] = '창신동'
df_child.loc[1128,'동'] = '갈현동'
df_child.loc[1129,'동'] = '대조동'
df_child.loc[1130,'동'] = '불광동'
df_child.loc[1146,'동'] = '하월곡동'
df_child.loc[1155,'동'] = '잠원동'
df_child.loc[1233,'동'] = '율현동'
df_child.loc[1321,'동'] = '명일동'
df_child.loc[1390,'동'] = '하왕십리동'
df_child.loc[1547,'동'] = '답십리동'
df_child.loc[1548,'동'] = '답십리동'
df_child.loc[1575,'동'] = '화곡동'
df_child.loc[1599,'동'] = '마곡동'
df_child.loc[1668,'동'] = '자곡동'
df_child.loc[1720,'동'] = '신공덕동'
df_child.loc[1799,'동'] = '신림동'
df_child.loc[1811,'동'] = '성산동'
df_child.loc[1830,'동'] = '장안동'
df_child.loc[1835,'동'] = '봉천동'
df_child.loc[1865,'동'] = '토정동'
df_child.loc[1917,'동'] = '삼선동4가'
df_child.loc[1920,'동'] = '봉천동'
df_child.loc[1934,'동'] = '구로동'
df_child.loc[1943,'동'] = '우면동'
df_child.loc[1946,'동'] = '옥수동'
df_child.loc[1998,'동'] = '창신동'
df_child.loc[2016,'동'] = '문정동'
df_child.loc[2130,'동'] = '잠실동'
df_child.loc[2159,'동'] = '풍납동'
df_child.loc[2161,'동'] = '신내동'
df_child.loc[2184,'동'] = '북가좌동'
df_child.loc[2249,'동'] = '가양동'
df_child.loc[2278,'동'] = '천왕동'
df_child.loc[2334,'동'] = '가락동'
df_child.loc[2424,'동'] = '상봉동'
df_child.loc[2469,'동'] = '신내동'
df_child.loc[2482,'동'] = '금호동3가'
df_child.loc[2523,'동'] = '도화동'
df_child.loc[2672,'동'] = '사당동'
df_child.loc[2688,'동'] = '상도동'
df_child.loc[2701,'동'] = '동빙고동'
df_child.loc[2703,'동'] = '잠실동'
df_child.loc[2726,'동'] = '신계동'
df_child.loc[2867,'동'] = '자양동'
df_child.loc[2969,'동'] = '신도림동'
df_child.loc[2978,'동'] = '길동'
df_child.loc[3163,'동'] = '상계동'
df_child.loc[3175,'동'] = '상계동'
df_child.loc[3387,'동'] = '상계동'
df_child.loc[3390,'동'] = '용산동3가'
df_child.loc[3447,'동'] = '하월곡동'
df_child.loc[3588,'동'] = '강일동'
df_child.loc[3689,'동'] = '남현동'
df_child.loc[3965,'동'] = '성내동'
df_child.loc[4021,'동'] = '신림동'
df_child.loc[4206,'동'] = '신정동'
df_child.loc[4355,'동'] = '내곡동'
df_child.loc[4359,'동'] = '봉천동'
df_child.loc[4369,'동'] = '대방동'
df_child.loc[4580,'동'] = '대치동'
df_child.loc[4749,'동'] = '대림동'
df_child.loc[4819,'동'] = '갈현동'
df_child.loc[4909,'동'] = '성수동1가'
df_child.loc[4910,'동'] = '행당동'
df_child.loc[6141,'동'] = '사당동'
df_child.loc[6262,'동'] = '이문동'
df_child.loc[6403,'동'] = '을지로1가'
df_child.loc[6681,'동'] = '봉천동'
df_child.loc[6722,'동'] = '답십리동'
df_child.loc[6886,'동'] = '신림동'
df_child.loc[7086,'동'] = '가락동'
df_child.loc[7122,'동'] = '전농동'
df_child.loc[7193,'동'] = '신도림동'
df_child.loc[7249,'동'] = '서소문동'
df_child.loc[7754,'동'] = '봉천동'
df_child.loc[7970,'동'] = '봉천동'
df_child.loc[7995,'동'] = '개봉동'
df_child.loc[8182,'동'] = '한남동'
df_child.loc[8299,'동'] = '사당동'
df_child.loc[8519,'동'] = '신림동'
df_child.loc[8648,'동'] = '방학동'
df_child.loc[8696,'동'] = '개봉동'
df_child.loc[9125,'동'] = '신정동'
df_child.loc[9200,'동'] = '고덕동'

In [ ]:
# CCTV
for idx in range(len(df_cctv['지번주소'])):
    if df_cctv['동'][idx] == '없음' and type(df_cctv['지번주소'][idx]) == str:
        for add in df_cctv['지번주소'][idx].split(' '):
            if len(add) > 0 and add[-1] == '동':
                df_cctv['동'][idx] = add
                break

for idx in range(len(df_cctv['지번주소'])):
    if df_cctv['동'][idx] == '없음' and type(df_cctv['소재지도로명주소'][idx]) == str:
        for add in df_cctv['소재지도로명주소'][idx].split(' '):
            if len(add) > 0 and add[-1] == '동':
                df_cctv['동'][idx] = add
                break

for idx in range(len(df_cctv['지번주소'])):
    if df_cctv['동'][idx] == '없음' and type(df_cctv['소재지도로명주소'][idx]) == str:
        count = 0
        for i in df_cctv['소재지도로명주소'][idx]:
            if i == '(' or i == '동':
                count += 1
            if count == 2:
                df_cctv['동'][idx] = df_cctv['소재지도로명주소'][idx][df_cctv['소재지도로명주소'][idx].index('(')+1:df_cctv['소재지도로명주소'][idx].index('동')+1]

df_cctv.loc[17, '동'] = '갈현동'
df_cctv.loc[101, '동'] = '신정동'
df_cctv.loc[136, '동'] = '증산동'
df_cctv.loc[160, '동'] = '금호동1가'
df_cctv.loc[161, '동'] = '금호동1가'
df_cctv.loc[241, '동'] = '행당동'
df_cctv.loc[485, '동'] = '영등포동'
df_cctv.loc[506, '동'] = '양화동'
df_cctv.loc[511, '동'] = '양평동2가'
df_cctv.loc[627, '동'] = '독산동'
df_cctv.loc[718, '동'] = '을지로3가'
df_cctv.loc[805, '동'] = '장충동2가'
df_cctv.loc[809, '동'] = '남대문로5가'
df_cctv.loc[879, '동'] = '필동2가'
df_cctv.loc[988, '동'] = '창천동'
df_cctv.loc[1019, '동'] = '현저동'
df_cctv.loc[1164, '동'] = '자양동'
df_cctv.loc[1170, '동'] = '구의동'
df_cctv.loc[1202, '동'] = '중곡동'
df_cctv.loc[1219, '동'] = '구의동'
df_cctv.loc[1238, '동'] = '화양동'
df_cctv.loc[1240, '동'] = '자양동'

for idx in range(1256, 1261):
  if df_cctv['동'][idx] == '없음':
    df_cctv.loc[idx, '동'] = '하계동'

df_cctv.loc[1261, '동'] = '구의동'
df_cctv.loc[1312, '동'] = '미아동'
df_cctv.loc[1390, '동'] = '미아동'
df_cctv.loc[1652, '동'] = '율현동'
df_cctv.loc[1653, '동'] = '율현동'
df_cctv.loc[1696, '동'] = '대치동'
df_cctv.loc[1768, '동'] = '개포동'
df_cctv.loc[1780, '동'] = '개포동'
df_cctv.loc[2100, '동'] = '전농동'
df_cctv.loc[2135, '동'] = '답십리동'
df_cctv.loc[2136, '동'] = '답십리동'
df_cctv.loc[2147, '동'] = '답십리동'
df_cctv.loc[2167, '동'] = '청량리동'
df_cctv.loc[2558, '동'] = '망우동'
df_cctv.loc[2605, '동'] = '신내동'
df_cctv.loc[2626, '동'] = '면목동'
df_cctv.loc[2632, '동'] = '망우동'
df_cctv.loc[2639, '동'] = '면목동'
df_cctv.loc[2642, '동'] = '망우동'
df_cctv.loc[2650, '동'] = '보문동3가'
df_cctv.loc[2758, '동'] = '궁동'
df_cctv.loc[2939, '동'] = '신대방동'
df_cctv.loc[3024, '동'] = '동작동'
df_cctv.loc[3375, '동'] = '둔촌동'
df_cctv.loc[3463, '동'] = '중계동'
df_cctv.loc[3492, '동'] = '중계동'
df_cctv.loc[3493, '동'] = '중계동'
df_cctv.loc[3497, '동'] = '중계동'
df_cctv.loc[3524, '동'] = '하계동'
df_cctv.loc[3530, '동'] = '중계동'
df_cctv.loc[3533, '동'] = '하계동'
df_cctv.loc[3534, '동'] = '하계동'
df_cctv.loc[3535, '동'] = '중계동'
df_cctv.loc[3537, '동'] = '중계동'
df_cctv.loc[3538, '동'] = '중계동'
df_cctv.loc[3539, '동'] = '중계동'
df_cctv.loc[3540, '동'] = '월계동'
df_cctv.loc[3541, '동'] = '월계동'
df_cctv.loc[3542, '동'] = '월계동'

for idx in range(3544, 3563):
  if df_cctv['동'][idx] == '없음':
    df_cctv.loc[idx, '동'] = '상계동'

df_cctv.loc[3563, '동'] = '공릉동'
df_cctv.loc[3568, '동'] = '상계동'
df_cctv.loc[3580, '동'] = '상계동'
df_cctv.loc[3581, '동'] = '월계동'
df_cctv.loc[3582, '동'] = '공릉동'
df_cctv.loc[3583, '동'] = '월계동'
df_cctv.loc[3608, '동'] = '공릉동'
df_cctv.loc[3627, '동'] = '중계동'
df_cctv.loc[3643, '동'] = '상계동'
df_cctv.loc[3654, '동'] = '상계동'

for idx in range(3660, 3665):
  if df_cctv['동'][idx] == '없음':
    df_cctv.loc[idx, '동'] = '하계동'

df_cctv.loc[3739, '동'] = '서초동'
df_cctv.loc[3740, '동'] = '서초동'
df_cctv.loc[3741, '동'] = '방배동'
df_cctv.loc[3743, '동'] = '봉천동'
df_cctv.loc[3744, '동'] = '내곡동'

for idx in range(4056, 4060):
  if df_cctv['동'][idx] == '없음':
    df_cctv.loc[idx, '동'] = '효창동'

df_cctv.loc[4066, '동'] = '이촌동'
df_cctv.loc[4078, '동'] = '용산동3가'
df_cctv.loc[4082, '동'] = '동빙고동'
df_cctv.loc[4176, '동'] = '명륜3가'
df_cctv.loc[4216, '동'] = '신문로2가'
df_cctv.loc[4271, '동'] = '신정동'
df_cctv.loc[4441, '동'] = '신월동'
df_cctv.loc[4443, '동'] = '목동'
df_cctv.loc[4523, '동'] = '예지동'

df_cctv.loc[229, '동'] = '성수동1가'
df_cctv.loc[415, '동'] = '보문동5가'
df_cctv.loc[918, '동']='인현동'
df_cctv.loc[986, '동'] = '홍제동'
df_cctv.loc[1313, '동'] = '미아동'
df_cctv.loc[2618, '동'] = '중화동'
df_cctv.loc[3143, '동'] = '화곡동'
df_cctv.loc[3183, '동'] = '가양동'
df_cctv.loc[3326, '동'] = '방화동'
df_cctv.loc[3327, '동'] = '방화동'
df_cctv.loc[3332, '동'] = '가양동'
df_cctv.loc[3475, '동'] = '상계동'
df_cctv.loc[3494, '동'] = '중계동'
df_cctv.loc[3495, '동'] = '중계동'
df_cctv.loc[3496, '동'] = '중계동'
df_cctv.loc[3499, '동'] = '중계동'
df_cctv.loc[3516, '동'] = '중계동'
df_cctv.loc[3531, '동'] = '월계동'
df_cctv.loc[3536, '동'] = '중계동'
df_cctv.loc[3543, '동'] = '월계동'
df_cctv.loc[3564, '동'] = '공릉동'
df_cctv.loc[3565, '동'] = '공릉동'
df_cctv.loc[3584, '동'] = '중계동'
df_cctv.loc[3585, '동'] = '공릉동'
df_cctv.loc[3586, '동'] = '중계동'
df_cctv.loc[3587, '동'] = '공릉동'
df_cctv.loc[3592, '동'] = '중계동'
df_cctv.loc[3593, '동'] = '중계동'
df_cctv.loc[3594, '동'] = '중계동'
df_cctv.loc[3595, '동'] = '공릉동'
df_cctv.loc[3588, '동'] = '상계동'
df_cctv.loc[3589, '동'] = '상계동'
df_cctv.loc[3590, '동'] = '상계동'
df_cctv.loc[3591, '동'] = '상계동'
df_cctv.loc[3596, '동'] = '상계동'
df_cctv.loc[3597, '동'] = '상계동'
df_cctv.loc[3598, '동'] = '상계동'
df_cctv.loc[3599, '동'] = '상계동'
df_cctv.loc[3600, '동'] = '공릉동'
df_cctv.loc[3601, '동'] = '월계동'
df_cctv.loc[3602, '동'] = '월계동'
df_cctv.loc[3603, '동'] = '월계동'
df_cctv.loc[3604, '동'] = '월계동'
df_cctv.loc[3645, '동'] = '상계동'
df_cctv.loc[3647, '동'] = '상계동'
df_cctv.loc[3648, '동'] = '상계동'
df_cctv.loc[3653, '동'] = '상계동'
df_cctv.loc[3655, '동'] = '상계동'
df_cctv.loc[3656, '동'] = '상계동'
df_cctv.loc[3657, '동'] = '상계동'
df_cctv.loc[4274, '동'] = '신정동'

In [ ]:
# 스타벅스
df_starbucks.loc[36, ['지번주소', '동']] = ['서울 강남구 신사동 529-10','신사동']
df_starbucks.loc[102, ['지번주소', '동']] = ['서울 강서구 마곡동 773-2','마곡동']
df_starbucks.loc[113, ['지번주소', '동']] = ['서울 강서구 화곡동 1013-3','화곡동']
df_starbucks.loc[115, ['지번주소', '동']] = ['서울 강서구 등촌동 628-9','등촌동']
df_starbucks.loc[116, ['지번주소', '동']] = ['서울 강서구 내발산동 668','내발산동']
df_starbucks.loc[117, ['지번주소', '동']] = ['서울 강서구 마곡동 759-2','마곡동']
df_starbucks.loc[142, ['지번주소', '동']] = ['서울 광징구 광장동 329-1','광장동']
df_starbucks.loc[163, ['지번주소', '동']] = ['서울 금천구 가산동 673','가산동']
df_starbucks.loc[185, ['지번주소', '동']] = ['서울 도봉구 방학동 707-6','방학동']
df_starbucks.loc[209, ['지번주소', '동']] = ['서울 마포구 대흥동 111-1','대흥동']
df_starbucks.loc[251, ['지번주소', '동']] = ['서울 서대문구 창천동 13-19','창천동']
df_starbucks.loc[311, ['지번주소', '동']] = ['서울 성북구 하월곡동 46-73','하월곡동']
df_starbucks.loc[336, ['지번주소', '동']] = ['서울 송파구 오금동 17-8','오금동']
df_starbucks.loc[360, ['지번주소', '동']] = ['서울 양천구 목동 406-4','목동']
df_starbucks.loc[368, ['지번주소', '동']] = ['서울 양천구 목동 920','목동']
df_starbucks.loc[370, ['지번주소', '동']] = ['서울 양천구 신정동 324-7','신정동']
df_starbucks.loc[388, ['지번주소', '동']] = ['서울 영등포구 신길동 252-9','신길동']
df_starbucks.loc[426, ['지번주소', '동']] = ['서울 은평구 녹번동 19-121','녹번동']
df_starbucks.loc[429, ['지번주소', '동']] = ['서울 종로구 동숭동 1-31','동숭동']
df_starbucks.loc[456, ['지번주소', '동']] = ['서울 종로구 동숭동 1-115','동숭동']
df_starbucks.loc[497, ['지번주소', '동']] = ['서울 중구 필동2가 19-1','필동2가']
df_starbucks.loc[531, ['지번주소', '동']] = ['서울 강동구 상일동 501','상일동']
df_starbucks.loc[572, ['지번주소', '동']] = ['서울 성동구 행당동 31-11','행당동']

In [ ]:
# 대학교
df_university_rev = df_university[df_university['동']=='없음']
df_university_rev.loc[:, '동'] = df_university_rev['주소'].str.split().str[2]
df_university_rev.loc[3, ['동']] = ['면목동']
df_university_rev.loc[23,['동']] = ['방이동']
df_university_rev.loc[37,['동']] = ['안암동5가']
df_university_rev.loc[39,['동']] = ['정릉동']
df_university_rev.loc[41,['동']] = ['화곡동']
df_university_rev.loc[44,['동']] = ['하월곡동']
df_university_rev.loc[56,['동']] = ['정릉동']


# 최종
df_university = pd.concat([df_university, df_university_rev], axis=0)
df_university.loc[37, '동'] = '휘경동'
df_university.loc[8, '동']='휘경동'

In [ ]:
# 초중고
df_school.loc[559, '동']='인현동'

# 결측치 처리 및 전처리한 데이터 불러오기

In [ ]:
# 법정동 주소가 없는 데이터(결측치 처리 후)
df_park=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/21년 서울시 공원(지번주소추가).csv', encoding='cp949')
df_mart=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_가공)/23년 서울시 마트(가공).csv')
df_bus=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 버스정류장(지번주소추가).csv')
df_child=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 어린이집(지번주소추가).csv', encoding='cp949')
df_cctv=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 CCTV 현황(지번주소추가).csv', encoding='cp949')
df_starbucks=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 스타벅스(지번주소추가).csv', encoding='cp949')
df_university=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/16년 서울시 소재 대학교(지번주소추가).csv', encoding='cp949')
df_hospital=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 종합병원(지번주소추가).csv', encoding='cp949')
df_school=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_지번주소 추가)/23년 서울시 학교(초중고)(지번주소추가).csv', encoding='cp949')

# 법정동 주소가 있는 데이터
df_subway=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세)/23년 서울시 지하철역.csv')
df_river=pd.read_csv('/content/drive/MyDrive/뽀로로드/data(아파트시세_가공)/23년 서울시 한강거리(가공).csv', encoding='cp949')

# 동별 Feature 갯수 계산

In [ ]:
def count_by_column(df, column_name):
    df_count = df['동'].value_counts().to_frame().reset_index()
    df_count = df_count.rename(columns={'동': column_name, 'index': '동'})
    return df_count

In [ ]:
# 공원
df_park_count = count_by_column(df_park, '공원')

# 버스정류장
df_bus_count = count_by_column(df_bus, '버스정류장')

# 어린이집
df_child_count = count_by_column(df_child, '어린이집')

# cctv
df_cctv_count = count_by_column(df_cctv, 'CCTV')

# 스타벅스
df_starbucks_count = count_by_column(df_starbucks, '스타벅스')

# 대학교
df_university_count = count_by_column(df_university, '대학교')

# 병원
df_hospital_count = count_by_column(df_hospital, '병원')

# 지하철
df_subway_count = count_by_column(df_subway, '지하철')

# 초중고
df_school_count = count_by_column(df_school, '초중고')

# 마트
df_mart_count = df_mart[['동', 'index']]
df_mart_count = df_mart_count.rename(columns={'동': '마트', 'index':'동'})

# 한강근처 동
df_river_count = df_river[['동', 'count']]
df_river_count = df_river_count.rename(columns={'count':'한강'})

<ipython-input-7-0848c3b5b815>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mart_count.drop(78, axis=0, inplace=True)


# feature merge

In [ ]:
merge_df1 = pd.merge(df_park_count, df_mart_count, on='동',how='outer')
merge_df2 = pd.merge(df_bus_count, df_child_count, on='동',how='outer')
merge_df3 = pd.merge(df_cctv_count, df_starbucks_count, on='동',how='outer')
merge_df4 = pd.merge(df_university_count, df_hospital_count, on='동',how='outer')
merge_df5 = pd.merge(df_subway_count, df_school_count, on='동',how='outer')

merge_df6 = pd.merge(merge_df1, merge_df2, on='동',how='outer')
merge_df7 = pd.merge(merge_df3, merge_df4, on='동',how='outer')
merge_df8 = pd.merge(merge_df5, df_river_count, on='동',how='outer')

merge_df = pd.merge(merge_df6, merge_df7, on='동',how='outer')
merge_df = pd.merge(merge_df, merge_df8, on='동',how='outer')

merge_df

,동,공원,마트,버스정류장,어린이집,CCTV,스타벅스,대학교,병원,지하철,초중고,한강
0,진관동,4.0,1.0,70.0,39.0,12.0,4.0,NaN,1.0,1.0,33.0,NaN
1,독산동,3.0,6.0,112.0,58.0,42.0,2.0,NaN,NaN,NaN,39.0,NaN
2,창동,3.0,1.0,112.0,70.0,26.0,4.0,NaN,NaN,4.0,19.0,NaN
3,면목동,3.0,2.0,123.0,58.0,38.0,1.0,1.0,1.0,3.0,15.0,NaN
4,잠실동,2.0,3.0,50.0,37.0,66.0,4.0,NaN,NaN,5.0,28.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
454,초동,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
455,합동,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
456,남학동,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
457,남산동3가,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
merge_df.fillna(0, inplace=True)
merge_df

,동,공원,마트,버스정류장,어린이집,CCTV,스타벅스,대학교,병원,지하철,초중고,한강
0,진관동,4.0,1.0,70.0,39.0,12.0,4.0,0.0,1.0,1.0,33.0,0.0
1,독산동,3.0,6.0,112.0,58.0,42.0,2.0,0.0,0.0,0.0,39.0,0.0
2,창동,3.0,1.0,112.0,70.0,26.0,4.0,0.0,0.0,4.0,19.0,0.0
3,면목동,3.0,2.0,123.0,58.0,38.0,1.0,1.0,1.0,3.0,15.0,0.0
4,잠실동,2.0,3.0,50.0,37.0,66.0,4.0,0.0,0.0,5.0,28.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
454,초동,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
455,합동,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
456,남학동,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
457,남산동3가,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
merge_df.to_csv("/content/drive/MyDrive/뽀로로드/data(아파트시세_코랩가공)/merge(확인용).csv", encoding='cp949')